In [ ]:
!pip install -q transformers datasets evaluate gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import random
import evaluate


In [ ]:
# Veriyi yükleme
data_path = "/content/drive/MyDrive/Colab Notebooks/NLP/medquad.csv"
df = pd.read_csv(data_path)

# Eksik verileri kontrol etme
print("Eksik veriler:", df.isnull().sum())

# Eksik verileri kaldırma
df_cleaned = df.dropna()
print(f"Temizlenmiş veri seti boyutu: {df_cleaned.shape}")


Eksik veriler: question       0
answer         5
source         0
focus_area    14
dtype: int64
Temizlenmiş veri seti boyutu: (16393, 4)


In [ ]:
# Tokenizer'ı yükleyelim
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

# distilgpt2 modelini yükleyelim
model = GPT2LMHeadModel.from_pretrained("distilgpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Token'lama fonksiyonu
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

# Dataset'i token'layalım
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Token'lanmış veriyi kontrol edelim
print(tokenized_datasets)


Map:   0%|          | 0/14753 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 14753
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1640
    })
})


In [ ]:
# Data collator oluşturma
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
# Eğitim parametrelerini ayarlama
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,  # Epoch sayısı
    per_device_train_batch_size=2,  # Eğitim için batch size
    per_device_eval_batch_size=2,  # Değerlendirme için batch size
    logging_dir="./logs",  # Loglar için dizin
    logging_steps=10,  # Her 10 adımda bir loglama
    eval_strategy="epoch",  # Değerlendirme stratejisi (epoch başına)
    save_strategy="epoch",  # Modeli her epoch sonunda kaydet
    save_total_limit=1,  # En fazla bir model kaydedilsin
    load_best_model_at_end=True,  # En iyi model sonrasında yüklensin
)


In [ ]:
# Trainer kurma
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-33-eae605f41a7a>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.800500,1.687437
2,1.518600,1.590867


Epoch,Training Loss,Validation Loss
1,1.800500,1.687437
2,1.518600,1.590867
3,1.625300,1.564448


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=22131, training_loss=1.7026215669751237, metrics={'train_runtime': 2588.0828, 'train_samples_per_second': 17.101, 'train_steps_per_second': 8.551, 'total_flos': 2891183221112832.0, 'train_loss': 1.7026215669751237, 'epoch': 3.0})

In [38]:
# Modeli kaydet
model_save_path = "./trained_model"
trainer.save_model(model_save_path)

# Tokenizer'ı da aynı klasöre kaydet
tokenizer.save_pretrained(model_save_path)


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json')

In [36]:
!pip install rouge_score
!pip install nltk


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5c3a29ad239a2d976b1e90268a2c4caf8c72d97218f91d67da90f4c4597753d8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [47]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import torch
import evaluate

# Metrikleri yükle
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

# Eğitilmiş modeli ve tokenizer'ı yükleyelim
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/NLP/trained_model")
tokenizer.pad_token = tokenizer.eos_token  # Pad token'ını eos_token olarak ayarla

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/NLP/trained_model")  # Eğitilmiş modeli yükle

# CSV verisini pandas ile yükleyelim
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/medquad.csv")

# Veriyi kontrol et ve NaN değerlerini temizle
df["question"] = df["question"].fillna("")  # NaN değerlerini boş string ile değiştir
df["answer"] = df["answer"].fillna("")  # NaN değerlerini boş string ile değiştir

# Tokenize etme fonksiyonu
def tokenize_function(examples):
    return tokenizer(examples["question"] + " " + examples["answer"], truncation=True, padding="max_length", max_length=256)

# Veriyi tokenize edelim
tokenized_dataset = []
for idx, row in df.iterrows():
    # NaN kontrolü sonrası, question ve answer birleştir
    text = str(row["question"]) + " " + str(row["answer"])  # float değerleri str'e çevir
    tokenized_text = tokenizer(text, truncation=True, padding="max_length", max_length=256)
    tokenized_dataset.append(tokenized_text)

# Test verisi üzerinde tahminler üretme fonksiyonu
def generate_predictions(model, tokenizer, dataset, num_samples=50):
    model.eval()
    predictions = []
    references = []

    for i in range(min(num_samples, len(dataset))):
        input_ids = torch.tensor(dataset[i]["input_ids"]).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=50,  # Sadece 50 token'lık çıktı üretir
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7,
                pad_token_id=tokenizer.eos_token_id  # pad_token_id tanımlandı
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reference_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)

        predictions.append(generated_text)
        references.append(reference_text)

    return predictions, references

# Test verisi üzerinde tahmin al
predictions, references = generate_predictions(model, tokenizer, tokenized_dataset, num_samples=50)

# BLEU skoru
bleu_score = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
print("BLEU Score:", bleu_score)

# ROUGE skoru
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print("ROUGE Score:", rouge_score)

# METEOR skoru
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print("METEOR Score:", meteor_score)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU Score: {'bleu': 0.7935204838597255, 'precisions': [0.7950919006261361, 0.7940519691433211, 0.7930014282799429, 0.7919401148482362], 'brevity_penalty': 1.0, 'length_ratio': 1.2572371762315897, 'translation_length': 9902, 'reference_length': 7876}
ROUGE Score: {'rouge1': np.float64(0.8661694947687164), 'rouge2': np.float64(0.8645160070502863), 'rougeL': np.float64(0.8662079522563142), 'rougeLsum': np.float64(0.8649970675168883)}
METEOR Score: {'meteor': np.float64(0.9111896865772984)}


In [49]:
!pip install gradio


In [52]:
import gradio as gr
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Modeli yükleyelim
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/NLP/trained_model")
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/NLP/trained_model")

# Tahmin fonksiyonu
def predict_answer(question):
    input_ids = tokenizer.encode(question, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    outputs = model.generate(input_ids, max_new_tokens=50, do_sample=True, top_k=50, top_p=0.95, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_answer

# Gradio arayüzü
iface = gr.Interface(fn=predict_answer, inputs="text", outputs="text", title="GPT Model", description="Ask a question .")

# Arayüzü başlat
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91ebfc0e45846a556d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
